In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List

'''
None || 20000 || 100000
'''
nb = None
nb_trains = nb
nb_valids = nb
nb_tests = nb

N_past = 10

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# cd drive/MyDrive/Projects/JPX_Tokyo_Stock/working

In [4]:
from data_utils import *

#### Load Data

In [5]:
train_df = pd.read_csv('../__input__/1_after_datapreprocessing/train_full.csv', parse_dates=['Date'], nrows = nb_trains)
valid_df = pd.read_csv('../__input__/1_after_datapreprocessing/valid_full.csv', parse_dates=['Date'], nrows = nb_valids)
test_df = pd.read_csv('../__input__/1_after_datapreprocessing/test_full.csv', parse_dates=['Date'], nrows = nb_tests)
y_test_df = pd.read_csv('../__input__/1_after_datapreprocessing/y_test_full.csv', parse_dates=['Date'], nrows = nb_tests)
official_test_df = DataPreprocessing_for_official_test(pd.read_csv('../__input__/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv', nrows = None, parse_dates=['Date']), train_df)

/mnt/ee1bd79c-6956-419b-951b-4699528687d7/Project/3. VDT miniPJ/working/data_utils.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_return['ExpectedDividend'] = df_return['ExpectedDividend'].fillna(-1)


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4617191 entries, 0 to 4617190
Data columns (total 13 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Date              datetime64[ns]
 1   SecuritiesCode    int64         
 2   Volume            int64         
 3   AdjustmentFactor  float64       
 4   ExpectedDividend  float64       
 5   SupervisionFlag   bool          
 6   Target            float64       
 7   CloseT2           float64       
 8   Open              float64       
 9   High              float64       
 10  Low               float64       
 11  Close             float64       
 12  CloseT1           float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory usage: 427.1 MB


In [7]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233748 entries, 0 to 233747
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              233748 non-null  datetime64[ns]
 1   SecuritiesCode    233748 non-null  int64         
 2   Volume            233748 non-null  int64         
 3   AdjustmentFactor  233748 non-null  float64       
 4   ExpectedDividend  233748 non-null  float64       
 5   SupervisionFlag   233748 non-null  bool          
 6   Target            233748 non-null  float64       
 7   CloseT2           233748 non-null  float64       
 8   Open              233748 non-null  float64       
 9   High              233748 non-null  float64       
 10  Low               233748 non-null  float64       
 11  Close             233748 non-null  float64       
 12  CloseT1           233748 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2)
memory 

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229952 entries, 0 to 229951
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Date              229952 non-null  datetime64[ns]
 1   SecuritiesCode    229952 non-null  int64         
 2   Volume            229952 non-null  int64         
 3   AdjustmentFactor  229952 non-null  float64       
 4   ExpectedDividend  229952 non-null  float64       
 5   SupervisionFlag   229952 non-null  bool          
 6   Open              229952 non-null  float64       
 7   High              229952 non-null  float64       
 8   Low               229952 non-null  float64       
 9   Close             229952 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2)
memory usage: 16.0 MB


In [9]:
y_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229952 entries, 0 to 229951
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   SecuritiesCode  229952 non-null  int64         
 1   Date            229952 non-null  datetime64[ns]
 2   CloseT1         229952 non-null  float64       
 3   CloseT2         229952 non-null  float64       
 4   Target          229952 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 8.8 MB


In [10]:
official_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              4000 non-null   datetime64[ns]
 1   SecuritiesCode    4000 non-null   int64         
 2   Volume            4000 non-null   int64         
 3   AdjustmentFactor  4000 non-null   float64       
 4   ExpectedDividend  4000 non-null   float64       
 5   SupervisionFlag   4000 non-null   bool          
 6   Open              4000 non-null   float64       
 7   High              4000 non-null   float64       
 8   Low               4000 non-null   float64       
 9   Close             4000 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(6), int64(2)
memory usage: 316.4 KB


In [11]:
train_df[train_df['SecuritiesCode']== 1301]

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Open,High,Low,Close,CloseT1
0,2017-01-04,1301,31400,1.0,-1.0,False,0.000730,2740.000000,2734.0,2755.0,2730.0,2742.0,2738.0
1863,2017-01-05,1301,17900,1.0,-1.0,False,0.002920,2748.000000,2743.0,2747.0,2735.0,2738.0,2740.0
3726,2017-01-06,1301,19900,1.0,-1.0,False,-0.001092,2738.007642,2734.0,2744.0,2720.0,2740.0,2741.0
5590,2017-01-10,1301,24200,1.0,-1.0,False,-0.005100,2731.000000,2745.0,2754.0,2735.0,2748.0,2745.0
7449,2017-01-11,1301,9300,1.0,-1.0,False,-0.003295,2722.000000,2748.0,2752.0,2737.0,2745.0,2731.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2314945,2021-11-29,1301,14300,1.0,-1.0,False,0.003793,2911.000000,2975.0,2984.0,2951.0,2951.0,2900.0
2316943,2021-11-30,1301,20500,1.0,-1.0,False,0.007558,2933.000000,2953.0,2997.0,2900.0,2900.0,2911.0
2318937,2021-12-01,1301,10600,1.0,-1.0,False,0.016706,2982.000000,2909.0,2936.0,2909.0,2911.0,2933.0
2320932,2021-12-02,1301,15200,1.0,-1.0,False,-0.003689,2971.000000,2949.0,2973.0,2933.0,2933.0,2982.0


#### Feature Engineering



- Diff between High and Low: Range_HL
- Diff between Open and Close: Range_OC


In [12]:
def add_range_hl_oc(df: pd.DataFrame):
    df['Range_HL'] = df['High'] - df['Low']
    df['Range_OC'] = df['Open'] - df['Close']
    df.drop(['High', 'Low', 'Open'], axis = 1, inplace= True)

In [13]:
add_range_hl_oc(train_df)
add_range_hl_oc(valid_df)
add_range_hl_oc(test_df)
add_range_hl_oc(official_test_df)

In [14]:
train_df

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,CloseT1,Range_HL,Range_OC
0,2017-01-04,1301,31400,1.0,-1.0,False,0.000730,2740.000000,2742.0,2738.0,25.0,-8.0
1,2017-01-04,1332,2798500,1.0,-1.0,False,0.012324,575.000000,571.0,568.0,13.0,-3.0
2,2017-01-04,1333,270800,1.0,-1.0,False,0.006154,3270.000000,3210.0,3250.0,70.0,-60.0
3,2017-01-04,1376,11300,1.0,-1.0,False,0.011053,1555.000000,1550.0,1538.0,40.0,-40.0
4,2017-01-04,1377,150800,1.0,-1.0,False,0.003026,3315.000000,3330.0,3305.0,80.0,-60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4617186,2021-12-03,9980,44400,1.0,-1.0,False,0.007692,131.000000,130.0,130.0,2.0,-1.0
4617187,2021-12-03,9986,3800,1.0,-1.0,False,0.004213,1903.987362,1896.0,1896.0,9.0,4.0
4617188,2021-12-03,9995,43400,1.0,-1.0,False,0.030534,406.030534,394.0,394.0,7.0,-4.0
4617189,2021-12-03,9996,500,1.0,-1.0,False,0.022150,1523.003257,1500.0,1490.0,10.0,-10.0


#### add date's features

In [15]:
def add_features_with_Date(df: pd.DataFrame):
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['week'] = df['Date'].dt.isocalendar().week
    df['is_month_end'] = df['Date'].dt.is_month_end
    df['is_month_start'] = df['Date'].dt.is_month_start
    df['is_year_end'] = df['Date'].dt.is_year_end
    df['is_year_start'] = df['Date'].dt.is_year_start   
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day

In [16]:
add_features_with_Date(train_df)
add_features_with_Date(valid_df)
add_features_with_Date(test_df)
add_features_with_Date(official_test_df)

Log volumn

In [17]:
train_df['Volume'] = np.log(train_df['Volume'])
valid_df['Volume'] = np.log(valid_df['Volume'])
test_df['Volume'] = np.log(test_df['Volume'])
official_test_df['Volume'] = np.log(official_test_df['Volume'])

/home/kienanh/miniconda3/envs/env/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
train_df.head(20)

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,CloseT1,...,dayofyear,dayofweek,week,is_month_end,is_month_start,is_year_end,is_year_start,Year,Month,Day
0,2017-01-04,1301,10.354563,1.0,-1.0,False,0.000730,2740.0,2742.0,2738.0,...,4,2,1,False,False,False,False,2017,1,4
1,2017-01-04,1332,14.844594,1.0,-1.0,False,0.012324,575.0,571.0,568.0,...,4,2,1,False,False,False,False,2017,1,4
2,2017-01-04,1333,12.509136,1.0,-1.0,False,0.006154,3270.0,3210.0,3250.0,...,4,2,1,False,False,False,False,2017,1,4
3,2017-01-04,1376,9.332558,1.0,-1.0,False,0.011053,1555.0,1550.0,1538.0,...,4,2,1,False,False,False,False,2017,1,4
4,2017-01-04,1377,11.923710,1.0,-1.0,False,0.003026,3315.0,3330.0,3305.0,...,4,2,1,False,False,False,False,2017,1,4
5,2017-01-04,1379,11.255449,1.0,-1.0,False,0.005169,2139.0,2143.0,2128.0,...,4,2,1,False,False,False,False,2017,1,4
6,2017-01-04,1381,7.170120,1.0,-1.0,False,-0.009326,1912.0,1960.0,1930.0,...,4,2,1,False,False,False,False,2017,1,4
7,2017-01-04,1407,11.898188,1.0,-1.0,False,-0.003437,882.0,866.0,885.0,...,4,2,1,False,False,False,False,2017,1,4
8,2017-01-04,1414,11.691908,1.0,-1.0,False,0.000000,5030.0,5050.0,5030.0,...,4,2,1,False,False,False,False,2017,1,4
9,2017-01-04,1417,12.758520,1.0,-1.0,False,-0.007463,1064.0,1053.0,1072.0,...,4,2,1,False,False,False,False,2017,1,4


Get info of N days before; calc mean and std of N days

In [19]:
def get_N_days(_df: pd.DataFrame, N, df_train= None, interpolate:bool = False):
    df = _df.copy()
    info_cols = ['Close', 'Range_HL', 'Range_OC', 'Volume']

    if df_train is None:
        df_train = pd.DataFrame(columns= ['SecuritiesCode', 'Date'] + info_cols)
    else:
        df_train = df_train[['SecuritiesCode', 'Date'] + info_cols]

    df_root = pd.DataFrame()
    for code in df_train['SecuritiesCode'].unique():
        df_root = df_root.append(
            df_train[
                df_train['SecuritiesCode'] == code
            ].sort_values(by=['Date'], ascending= True)[-N:]
        )

    for i in range(N):
        tmp = df_root.append(df[['SecuritiesCode', 'Date'] + info_cols])
        tmp.columns = ['SecuritiesCode', 'Date'] + [col_name + '_' + str(i + 1) +'_before' for col_name in info_cols]

        sorted_tmp = pd.DataFrame()
        for code in tmp['SecuritiesCode'].unique():

            df_tmp = tmp[tmp['SecuritiesCode'] == code]
            df_tmp = df_tmp.sort_values(by=['Date'], ascending= True)

            df_tmp['Date'] = df_tmp['Date'].shift(-(i + 1))
            sorted_tmp = sorted_tmp.append(df_tmp)

        df = df.merge(
            sorted_tmp,
            how='left', on=['SecuritiesCode', 'Date']
        )

        if interpolate:
            df = fillInterpolate(df, [col_name + '_' + str(i + 1) +'_before' for col_name in info_cols])
    if not interpolate:
        df.dropna(inplace= True)
    
    df = df.reset_index(drop= True)
    for col_name in ['Close']:
        df[col_name + '_mean'] = df[[col_name] + [col_name + '_' + str(i + 1) +'_before' for i in range(N)]].mean(axis= 1)
        df[col_name + '_std'] = df[[col_name] + [col_name + '_' + str(i + 1) +'_before' for i in range(N)]].std(axis= 1).replace(0, method='bfill')
    return df

In [20]:
train_df = get_N_days(train_df, N= N_past, interpolate= False)
valid_df = get_N_days(valid_df, N= N_past, df_train= train_df, interpolate= False)
test_df = get_N_days(test_df, N= N_past, df_train= train_df, interpolate= True)
official_test_df = get_N_days(official_test_df, N= N_past, df_train= train_df, interpolate= True)

In [21]:
train_df

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,CloseT1,...,Close_9_before,Range_HL_9_before,Range_OC_9_before,Volume_9_before,Close_10_before,Range_HL_10_before,Range_OC_10_before,Volume_10_before,Close_mean,Close_std
0,2017-01-19,1301,9.487972,1.0,-1.0,False,-0.006211,2720.000000,2699.0,2737.0,...,2738.0,12.0,5.0,9.792556,2742.0,25.0,-8.0,10.354563,2722.636364,22.791545
1,2017-01-19,1332,14.963103,1.0,-1.0,False,-0.018349,535.000000,554.0,545.0,...,568.0,8.0,4.0,14.586960,571.0,13.0,-3.0,14.844594,552.272727,16.613795
2,2017-01-19,1333,12.091223,1.0,-1.0,False,-0.007837,3165.000000,3245.0,3190.0,...,3250.0,70.0,-25.0,12.715597,3210.0,70.0,-60.0,12.509136,3222.727273,32.121927
3,2017-01-19,1376,8.648221,1.0,-1.0,False,-0.006614,1502.000000,1496.0,1512.0,...,1538.0,102.0,12.0,9.517825,1550.0,40.0,-40.0,9.332558,1531.909091,25.315823
4,2017-01-19,1377,12.296827,1.0,-1.0,False,-0.012121,3260.000000,3310.0,3300.0,...,3305.0,60.0,35.0,11.955686,3330.0,80.0,-60.0,11.923710,3303.636364,48.015149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4573504,2021-12-03,9980,10.700995,1.0,-1.0,False,0.007692,131.000000,130.0,130.0,...,133.0,5.0,4.0,12.112860,137.0,2.0,1.0,11.140411,131.454545,2.296242
4573505,2021-12-03,9986,8.242756,1.0,-1.0,False,0.004213,1903.987362,1896.0,1896.0,...,1857.0,6.0,3.0,7.972466,1863.0,12.0,11.0,7.937375,1858.090909,19.387906
4573506,2021-12-03,9995,10.678215,1.0,-1.0,False,0.030534,406.030534,394.0,394.0,...,412.0,5.0,-2.0,10.451609,408.0,5.0,-1.0,10.589106,401.818182,9.693484
4573507,2021-12-03,9996,6.214608,1.0,-1.0,False,0.022150,1523.003257,1500.0,1490.0,...,1500.0,2.0,0.0,6.551080,1500.0,0.0,0.0,6.396930,1496.000000,8.955445


Scale data: mean = 0; std = 1

In [22]:
def ScaleData(df: pd.DataFrame, N):
    for col_name in ['Close']:
        df[col_name] = (df[col_name] - df['Close_mean'])/df['Close_std']
        for i in range(N):
            name = col_name + '_' + str(i + 1) +'_before'
            df[name] = (df[name] - df['Close_mean'])/df['Close_std']

    for col_name in ['Range_HL', 'Range_OC']:
        df[col_name] = df[col_name]/df['Close_std']
        for i in range(N):
            name = col_name + '_' + str(i + 1) +'_before'
            df[name] = df[name]/df['Close_std']

In [23]:
ScaleData(train_df, N_past)
ScaleData(valid_df, N_past)
ScaleData(test_df, N_past)
ScaleData(official_test_df, N_past)

In [24]:
train_df['CloseT1'] = (train_df['CloseT1'] - train_df['Close_mean'])/train_df['Close_std']
train_df['CloseT2'] = (train_df['CloseT2'] - train_df['Close_mean'])/train_df['Close_std']

valid_df['CloseT1'] = (valid_df['CloseT1'] - valid_df['Close_mean'])/valid_df['Close_std']
valid_df['CloseT2'] = (valid_df['CloseT2'] - valid_df['Close_mean'])/valid_df['Close_std']

y_test_df['CloseT1'] = (y_test_df['CloseT1'] - test_df['Close_mean'])/test_df['Close_std']
y_test_df['CloseT2'] = (y_test_df['CloseT2'] - test_df['Close_mean'])/test_df['Close_std']

#### Save and Check

In [25]:
train_df.to_csv('../__input__/2_after_featureEgineering/train_full.csv', index = False)
valid_df.to_csv('../__input__/2_after_featureEgineering/valid_full.csv', index = False)
test_df.to_csv('../__input__/2_after_featureEgineering/test_full.csv', index = False)
y_test_df.to_csv('../__input__/2_after_featureEgineering/y_test_full.csv', index = False)

In [26]:
train_df.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4573509 entries, 0 to 4573508
Data columns (total 64 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Date                4573509 non-null  datetime64[ns]
 1   SecuritiesCode      4573509 non-null  int64         
 2   Volume              4573509 non-null  float64       
 3   AdjustmentFactor    4573509 non-null  float64       
 4   ExpectedDividend    4573509 non-null  float64       
 5   SupervisionFlag     4573509 non-null  bool          
 6   Target              4573509 non-null  float64       
 7   CloseT2             4573509 non-null  float64       
 8   Close               4573509 non-null  float64       
 9   CloseT1             4573509 non-null  float64       
 10  Range_HL            4573509 non-null  float64       
 11  Range_OC            4573509 non-null  float64       
 12  dayofyear           4573509 non-null  int64         
 13  dayofweek   

In [27]:
train_df.head()

,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,CloseT1,...,Close_9_before,Range_HL_9_before,Range_OC_9_before,Volume_9_before,Close_10_before,Range_HL_10_before,Range_OC_10_before,Volume_10_before,Close_mean,Close_std
0,2017-01-19,1301,9.487972,1.0,-1.0,False,-0.006211,-0.115673,-1.037067,0.630218,...,0.674094,0.526511,0.219380,9.792556,0.849597,1.096898,-0.351007,10.354563,2722.636364,22.791545
1,2017-01-19,1332,14.963103,1.0,-1.0,False,-0.018349,-1.039662,0.103966,-0.437752,...,0.946639,0.481528,0.240764,14.586960,1.127212,0.782482,-0.180573,14.844594,552.272727,16.613795
2,2017-01-19,1333,12.091223,1.0,-1.0,False,-0.007837,-1.797130,0.693381,-1.018845,...,0.849038,2.179197,-0.778285,12.715597,-0.396218,2.179197,-1.867883,12.509136,3222.727273,32.121927
3,2017-01-19,1376,8.648221,1.0,-1.0,False,-0.006614,-1.181439,-1.418445,-0.786429,...,0.240597,4.029101,0.474012,9.517825,0.714609,1.580039,-1.580039,9.332558,1531.909091,25.315823
4,2017-01-19,1377,12.296827,1.0,-1.0,False,-0.012121,-0.908804,0.132534,-0.075734,...,0.028400,1.249606,0.728937,11.955686,0.549069,1.666141,-1.249606,11.923710,3303.636364,48.015149


In [28]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233223 entries, 0 to 233222
Data columns (total 64 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Date                233223 non-null  datetime64[ns]
 1   SecuritiesCode      233223 non-null  int64         
 2   Volume              233223 non-null  float64       
 3   AdjustmentFactor    233223 non-null  float64       
 4   ExpectedDividend    233223 non-null  float64       
 5   SupervisionFlag     233223 non-null  bool          
 6   Target              233223 non-null  float64       
 7   CloseT2             233223 non-null  float64       
 8   Close               233223 non-null  float64       
 9   CloseT1             233223 non-null  float64       
 10  Range_HL            233223 non-null  float64       
 11  Range_OC            233223 non-null  float64       
 12  dayofyear           233223 non-null  int64         
 13  dayofweek           233223 no

In [29]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229952 entries, 0 to 229951
Data columns (total 61 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Date                229952 non-null  datetime64[ns]
 1   SecuritiesCode      229952 non-null  int64         
 2   Volume              229952 non-null  float64       
 3   AdjustmentFactor    229952 non-null  float64       
 4   ExpectedDividend    229952 non-null  float64       
 5   SupervisionFlag     229952 non-null  bool          
 6   Close               229952 non-null  float64       
 7   Range_HL            229952 non-null  float64       
 8   Range_OC            229952 non-null  float64       
 9   dayofyear           229952 non-null  int64         
 10  dayofweek           229952 non-null  int64         
 11  week                229952 non-null  UInt32        
 12  is_month_end        229952 non-null  bool          
 13  is_month_start      229952 no

In [30]:
official_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 61 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                4000 non-null   datetime64[ns]
 1   SecuritiesCode      4000 non-null   int64         
 2   Volume              4000 non-null   float64       
 3   AdjustmentFactor    4000 non-null   float64       
 4   ExpectedDividend    4000 non-null   float64       
 5   SupervisionFlag     4000 non-null   bool          
 6   Close               4000 non-null   float64       
 7   Range_HL            4000 non-null   float64       
 8   Range_OC            4000 non-null   float64       
 9   dayofyear           4000 non-null   int64         
 10  dayofweek           4000 non-null   int64         
 11  week                4000 non-null   UInt32        
 12  is_month_end        4000 non-null   bool          
 13  is_month_start      4000 non-null   bool        

#### Mini Trainset

In [31]:
miniset = train_df[train_df['Date'] > '2021-11-01'].reset_index()
miniset

,index,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,...,Close_9_before,Range_HL_9_before,Range_OC_9_before,Volume_9_before,Close_10_before,Range_HL_10_before,Range_OC_10_before,Volume_10_before,Close_mean,Close_std
0,2261056,2021-11-02,1301,9.786954,1.0,-1.0,False,-0.043077,6.515019,1.536235,...,0.071886,1.610783,1.317914,8.881836,0.657626,1.025044,-0.439305,8.699515,3027.545455,34.144878
1,2261057,2021-11-02,1332,13.944343,1.0,-1.0,False,0.040945,-2.707243,-0.017466,...,-0.209593,4.034665,2.497650,14.093672,1.135295,1.729142,0.384254,14.217570,649.090909,5.204893
2,2261058,2021-11-02,1333,11.575900,1.0,-1.0,False,-0.001917,0.213653,0.412259,...,0.677068,1.588853,0.959932,11.143310,1.703202,0.761325,0.562719,10.918718,2601.545455,30.210474
3,2261059,2021-11-02,1375,10.740735,1.0,-1.0,False,-0.015972,-0.042552,-0.711221,...,0.759851,1.537939,1.069870,11.747207,2.565257,1.270471,-0.468068,10.956056,1440.636364,14.955084
4,2261060,2021-11-02,1376,7.244228,1.0,-1.0,False,-0.010554,1.369069,1.093586,...,-0.329745,0.596881,-0.229570,7.824046,-0.559315,0.413225,-0.367311,7.170120,1486.181818,21.779891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88231,4573504,2021-12-03,9980,10.700995,1.0,-1.0,False,0.007692,-0.197952,-0.633446,...,0.673036,2.177471,1.741977,12.112860,2.415013,0.870988,0.435494,11.140411,131.454545,2.296242
88232,4573505,2021-12-03,9986,8.242756,1.0,-1.0,False,0.004213,2.367272,1.955296,...,-0.056268,0.309471,0.154736,7.972466,0.253204,0.618943,0.567364,7.937375,1858.090909,19.387906
88233,4573506,2021-12-03,9995,10.678215,1.0,-1.0,False,0.030534,0.434555,-0.806540,...,1.050378,0.515810,-0.206324,10.451609,0.637729,0.515810,-0.103162,10.589106,401.818182,9.693484
88234,4573507,2021-12-03,9996,6.214608,1.0,-1.0,False,0.022150,3.015289,0.446656,...,0.446656,0.223328,0.000000,6.551080,0.446656,0.000000,0.000000,6.396930,1496.000000,8.955445


In [32]:
miniset[miniset['SecuritiesCode'] == 1301]

,index,Date,SecuritiesCode,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,CloseT2,Close,...,Close_9_before,Range_HL_9_before,Range_OC_9_before,Volume_9_before,Close_10_before,Range_HL_10_before,Range_OC_10_before,Volume_10_before,Close_mean,Close_std
0,2261056,2021-11-02,1301,9.786954,1.0,-1.0,False,-0.043077,6.515019,1.536235,...,0.071886,1.610783,1.317914,8.881836,0.657626,1.025044,-0.439305,8.699515,3027.545455,34.144878
1994,2263050,2021-11-04,1301,11.077516,1.0,-1.0,False,-0.014469,0.255264,2.705559,...,-0.473202,0.397345,0.264897,8.764053,-0.208305,0.728466,0.596018,8.881836,3045.727273,75.501114
3988,2265044,2021-11-05,1301,9.846917,1.0,-1.0,False,-0.011419,0.022078,0.733998,...,-0.553717,0.450700,0.000000,9.035987,-0.553717,0.386314,0.257543,8.764053,3053.000000,77.656938
5983,2267039,2021-11-08,1301,9.239899,1.0,-1.0,False,0.006601,-0.104751,0.091657,...,-0.890385,0.209502,0.065469,9.126959,-0.628507,0.458286,0.000000,9.035987,3058.000000,76.371461
7979,2269035,2021-11-09,1301,8.999619,1.0,-1.0,False,-0.001639,-0.196669,-0.395752,...,-0.900094,0.530887,-0.159266,9.190138,-0.926639,0.212355,0.066361,9.126959,3059.818182,75.345628
9971,2271027,2021-11-10,1301,8.536996,1.0,-1.0,False,0.021346,0.622268,-0.212482,...,-0.768982,0.403463,-0.333900,9.210340,-1.019407,0.556500,-0.166950,9.190138,3065.272727,71.877800
11964,2273020,2021-11-11,1301,8.648221,1.0,-1.0,False,-0.004823,0.365478,-0.368145,...,-1.087096,1.012400,0.205414,11.287279,-0.881681,0.425501,-0.352139,9.210340,3070.090909,68.154904
13952,2275008,2021-11-12,1301,9.667765,1.0,-1.0,False,0.008078,0.886121,0.467137,...,-0.442333,0.879155,-0.803366,9.277999,-1.260857,1.045891,0.212210,11.287279,3079.181818,65.972446
15948,2277004,2021-11-15,1301,8.922658,1.0,-1.0,False,-0.019231,-0.469934,0.113694,...,-0.053057,0.500253,0.000000,9.786954,-0.636685,0.967155,-0.883780,9.277999,3088.181818,59.969689
17941,2278997,2021-11-16,1301,9.011889,1.0,-1.0,False,-0.004902,-0.836602,0.429814,...,-0.245608,0.590994,0.084428,9.786954,-0.161180,0.506567,0.000000,9.786954,3094.545455,59.222232


In [33]:
miniset.to_csv('../__input__/2_after_featureEgineering/mini_train.csv', index = False)